# Linear Transformation
We will start with the alignement of the same models for different seeds.
- First we do not restrict the matrix.
- Second we restrict the matrix to be a rotation matrix.
- Thrid use affine transformation
- Last but not least we are using different norms and regularization techniques to improve the results



Steps:
- Load the same model but with different seed
- Sample different images and get latent representation
- Create Datamatrix X and X'
- Solve the simple optimization problem (Using closed form solution as well cvxpy solver)


In [88]:
#Amount google driv
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/case_study_opti/latent-communication/optimization'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

Mounted at /content/gdrive
['ResNet-LinearTransform', 'VAE-LinearTransform', '__pycache__', 'linear.ipynb', 'linear.py', 'linear_hillary.ipynb', 'nonlinear.ipynb', 'optimizer.py', 'utils', 'visualization.ipynb']


In [89]:
!pip install lightning

In [90]:
# Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
import cvxpy as cp
import os
from optimizer import AffineFitting, LinearFitting
from utils.sampler import *
os.chdir('../')
print(f"Current working directory {os.getcwd()}")

Current working directory /content/gdrive/MyDrive/case_study_opti/latent-communication


In [186]:
# Configuration
seed1 = 3
seed2 = 4
# 12, 13, 14, 23, 24 ,34
config = {
    'path1': os.getcwd()+f"/resnet/models/model_seed{seed1}.pth",
    'modelname1': 'resnet',
    'seed1': f'{seed1}',
    'path2': os.getcwd()+f"/resnet/models/model_seed{seed2}.pth",
    'modelname2': 'resnet',
    'seed2': f'{seed2}',
    'num_samples': '100',
    'storage_path': 'ResNet-LinearTransform'

}

## Load Model and Transformed Data
In this section we load the trained models, which we prepaired for our experimental setup

In [187]:
# Import Data
from helper.DataLoaderMNIST import DataLoader_MNIST
# Transformations
transformations = [transforms.ToTensor(),
                    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
                    ]
# Load the data
data_loader = DataLoader_MNIST(128, transformations)

In [188]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [189]:
from resnet.model_def import ResNet
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the models
model = ResNet(pretrained=False).to(DEVICE)
model.load_state_dict(torch.load(config['path1'],map_location = DEVICE))


# Initialize the model 2
model2 = ResNet(pretrained=False).to(DEVICE)# Load pretrained weights for model2
model2.load_state_dict(torch.load(config['path2'],map_location = DEVICE))

<All keys matched successfully>

## Sampling
We sample images from the train set and encode those for each model. In this section we implemented two sampler.
One just samples randomly and the second samples accordingly to the classes.

In [132]:
num_samples = int(config['num_samples'])
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
z1,z2,images,labels = simple_sampler(num_samples, model, model2, data_loader, DEVICE)

## Optimization

### Linear Transformation

In [190]:
#Do Linear fitting
lamda = 0.01
linear_fitting = LinearFitting(z1, z2, lamda)
linear_fitting.solve_problem()

Solving the problem
Defining the problem


In [191]:
name = 'Linear_' + config['modelname1'] + '_' + config['seed1'] + '_' + config['modelname2'] + '_' + config['seed2'] + '_' + config['num_samples']
path = 'optimization/' + config['storage_path'] + '/' + name

In [192]:
#linear_fitting.save_results(path) # Does not work in google colab

In [193]:
_,A = linear_fitting.get_results()
np.save(path, A)

### Affine Transformation

In [194]:
lamda=0.01
# Create an instance of the AffineFitting class
affine_fitting = AffineFitting(z1, z2, lamda)
# Solve the optimization problem
affine_fitting.solve_problem()

Solving the problem
Defining the problem


In [195]:
# Print the results
name = 'Affine_' + config['modelname1'] + '_' + config['seed1'] + '_' + config['modelname2'] + '_' + config['seed2'] + '_' + config['num_samples']
path = 'optimization/' + config['storage_path'] + '/' + name

In [196]:
#linear_fitting.save_results(path) #does not work in google colab

In [197]:
_,A,b = affine_fitting.get_results()
np.savez(path, A=A,b=b)

Getting results
